In [1]:
import os
from transformers import pipeline
import json
import numpy as np
import pandas as pd
VIDEOS = ['jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp',
 'JFBB_Rhythm_Section_Day1_Morning Thomas',
 'JFBB_Rhythm_Section_Day1_Afternoon Thomas',
 'JFBB_Jazz_Funk_morning_video20230419101818 Luis Cal García',
 'JFBB_Jazz_Funk_afternoon_video20230419143359 Luis Cal García',
 'jazzaar-20230417-afternoon-JFLB-Cuban-Vocals-video Philipp',
 'jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp']

/home/bsauter/anaconda3/envs/aaWorkshop/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-04 18:08:17.763972: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-04 18:08:18.781043: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-04 18:08:18.791768: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 18:08:21.749073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
columns = ['segment_id', 'start', 'end', 'text']
dimensions = (-1,4)
for video in VIDEOS:
    output = []
    for root, dir, files in os.walk(f"../data/transcriptions/processed/{video}"):
        for file in files:
            
            with open(f'{root}/{file}') as f:
                text = f.read()
                f.close()

            
            meta_info = [file.split('_')[i] for i in range(3)]
            output = output + meta_info + [text]
            
    output = np.reshape(output, dimensions)
    output_df = pd.DataFrame(output, columns=columns)

    
    output_df=output_df.astype({'start': 'int32', 'end': 'int32', 'text': 'string'})
    output_df=output_df.sort_values(by='start', axis=0)

    output_df['end_l1'] = output_df['end'].shift(1)
    output_df['diff_to_last_snippet'] = output_df['start'] - output_df['end_l1']
    curr_row = output_df.iloc[0]
    rows =[]
    for index, row in output_df.iterrows():
        if row.segment_id == '1':
            continue
        elif row.diff_to_last_snippet < 20:
            curr_row.end = row.end
            curr_row.text = curr_row.text + row.text
        else:
            
            if not os.path.exists(f"../data/transcriptions/processed_merged/{video}"):
                        # Create a new directory because it does not exist
                os.makedirs(f"../data/transcriptions/processed_merged/{video}")

            with open(f'../data/transcriptions//processed_merged/{video}/{curr_row.segment_id}_{curr_row.start}_{curr_row.end}_seg.txt', 'w') as output:
                output.write(curr_row.text)
            rows.append(curr_row)
            curr_row = row
        
    df=pd.DataFrame(rows)
    df['len'] = [len(text) for text in df['text'].to_numpy()]
    df.drop(['start','end','text','end_l1','diff_to_last_snippet'], axis=1, inplace=True)
    df.to_csv(f'../data/transcriptions/transcription_length/{video}.csv')